## Many thanks to Andrew W. Task and his Great book Grokking Deep Learning 

In [1]:
import numpy as np

In [201]:
class Tensor(object):
    
    def __init__(self, data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        if id is None:
            id = np.random.randint(0, 100000)
        self.id = id
        
        if creators is not None:
            for father in creators:
                if self.id not in father.children:
                    father.children[self.id] = 1
                else:
                    father.children[self.id] += 1
                    
    def all_children_grads_accounter_for(self):
        #in the backward function every time
        #we backprop through a children we decrease
        #the count, so we can confirme if the Tensor
        #has recived the correct number of grad
        
        for children_id, count in self.children.items():
            if count != 0:
                return False
        return True
        
        
    def backward(self, grad, grad_origin=None):
        if self.autograd:
            if grad_origin is not None:
                if self.children[grad_origin.id] == 0:
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
                    
            if self.grad is None:
                if type(grad) == np.ndarray:
                    self.grad = grad.copy()
                else:
                    self.grad = grad
            else:
                self.grad += grad

            if self.creators is not None and (self.all_children_grads_accounter_for() or grad_origin is None):
                
                if(self.creation_op == "add"):
                    self.creators[0].backward(grad, self)
                    self.creators[1].backward(grad, self)
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())


        
    def __add__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data + other.data,
                         autograd=True,
                         creators=[self, other],
                         creation_op="add")
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if self.autograd:
            return Tensor(self.data * -1,
                         autograd=True,
                         creators=[self],
                         creation_op="neg")
        return Tensor(self.data * -1)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [202]:
a = Tensor([1, 2, 3, 4, 5], autograd=True)
b = Tensor([2, 2, 2, 2, 2], autograd=True)
c = Tensor([5, 4, 3, 2, 1], autograd=True)

In [203]:
d = a + b
e = b + c
f = d + e

In [205]:
f.backward(np.array([1, 1, 1, 1, 1]))

In [42]:
b.children

{388: 1, 32523: 1}

In [25]:
print("Creators:", z.creators, "\nCreation op:", z.creation_op,"\nZ Grad:", z.grad,
     "\nx Grad:", x.grad, "\ny Grad:", y.grad)

Creators: [array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])] 
Creation op: add 
Grad: [1 1 1 1 1]


In [22]:
z.backward(np.array([1, 1, 1, 1, 1]))

In [27]:
print("Creators:", z.creators, "\nCreation op:", z.creation_op,"\nZ Grad:", z.grad,
     "\nx Grad:", x.grad, "\ny Grad:", y.grad)

Creators: [array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])] 
Creation op: add 
Z Grad: [1 1 1 1 1] 
x Grad: [1 1 1 1 1] 
y Grad: [1 1 1 1 1]


In [28]:
w = z + x

In [29]:
w.backward(np.array([1, 1, 1, 1, 1]))

In [31]:
print("W Creators:", w.creators, "\nW Creation op:", w.creation_op,"\nZ Grad:", z.grad,
     "\nx Grad:", x.grad, "\ny Grad:", y.grad, "\nw Grad:", w.grad)

W Creators: [array([3, 4, 5, 6, 7]), array([1, 2, 3, 4, 5])] 
W Creation op: add 
Z Grad: [1 1 1 1 1] 
x Grad: [1 1 1 1 1] 
y Grad: [1 1 1 1 1] 
w Grad: [1 1 1 1 1]


In [32]:
a = {'a': 1, 'b':1}
for id, cnt in a.items():
    print(id, cnt)

a 1
b 1
